<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Нейронная сеть с нуля
### Modules - Modern
<section class="post-meta">
By Sondre Duna Lundemo, Jenny Lunde, Niels Henrik Aase, Thorvald M. Ballestad, Jon Andreas Støvneng and Brynjulf Owren.
</section>

Last edited: March 1st, 2021

---

Этот блокнот даст представление о том, как строится полносвязанная нейронная сеть и как работают различные компоненты. По всей записной книжке будут представлены фрагменты кода для каждого компонента в сети, чтобы легче увидеть связь между уравнениями и реализациями. Ближе к концу все компоненты будут собраны в классы, чтобы сделать код более функциональным и аккуратным. Задача основана на отрывке из курса TMA4320 - Введение в научные вычисления в НТНУ.

#### Замечание:
Если вы не знакомы с объектно-ориентированным программированием, не паникуйте! То, как классы используются в этой записной книжке, будет легко понято кем-то с небольшим опытом программирования; вы можете думать о том, что это практический способ сбора определенных переменных вместе с функциями, которые вы используете для управления ими. Если вы хотите узнать больше, вы можете, например, прочитать больше <a href="https://docs.python.org/3/tutorial/classes.html">здесь</a>.

# Наброски задачи

Использование нейронных сетей оказало большое влияние на проблемы, связанные с искусственным интеллектом. Общий характер метода позволяет ему превосходно выполнять самые разнообразные задачи, начиная от полезных приложений, таких как распознавание изображений и самоуправляемые автомобили, и заканчивая менее полезными приложениями, например, в видеоиграх (*Очень спорно*). Еще одним важным примером использования нейронных сетей является решение задач классификации. Такого рода проблемы также возникают в физических науках, поэтому использование нейронных сетей в некоторых случаях может также дать представление об этих областях, понимание, которое недоступно, если ограничиться обычными методами, используемыми в численном анализе. В будущем блокноте мы рассмотрим такую проблему, используя обширное оборудование, предоставляемое в различных пакетах Python для машинного обучения. Однако в данном случае мы будем реализовывать алгоритмы с нуля, чтобы получить представление о механике нейронной сети. 

Хотя проблема не обязательно должна заключаться в классификации изображений, чтобы сделать работу сети менее абстрактной, мы часто будем ссылаться на входные данные сети как на _image_. Мы представляем, что каждый пиксель изображения имеет скалярное значение, скажем, представляющее значение в оттенках серого. Чтобы избежать использования матриц в качестве представления входных данных, мы складываем строки изображения друг на друга, чтобы создать входной вектор, размеры которого обязательно будут произведением количества пикселей в каждом направлении изображения. В случае двоичной классификации метка, связанная с каждым изображением, равна либо $0$, либо $1$, и она представляет какую-то категорию. Если, например, проблема заключалась в том, чтобы определить, изображен ли на изображении волк или хаски, мы могли бы выбрать обозначение $0$ для категории волка, и $1$ , для категории хаски.

# Что такое искусственная нейронная сеть?

Искусственная нейронная сеть (artificial neural network - ANN) - это набор функций, которые объединяются для имитации биологической нейронной сети [[1]](#Biological_net). Точно так же, как биологическая нейронная сеть, ANN состоит из множества нейронов, соединенных вместе, образуя сложную сеть. В ANN нейроны структурированы слоями. Первый называется входным слоем, последний слой называется выходным слоем, и между ними есть несколько скрытых слоев$^1$. Количество скрытых слоев, также известное как глубина сети, будет варьироваться в зависимости от сложности проблемы, которую вы хотите решить с помощью сети. Каждый слой содержит нейроны, и количество нейронов варьируется от сети к сети, а также может варьироваться от слоя к слою в данной сети. 

<center>
    <img src="images/fully_connected.PNG" alt="Fully Connected Neural Network" style="width: 500px;">
    <i>Этот рисунок представляет собой визуализацию сети, сделанной в этом блокноте. Сеть полностью подключена, и количество нейронов одинаково для всех слоев, кроме выходного слоя.</i>
    <img src="images/not_fully_connected.PNG" alt="Not fully Connected Neural Network" style="width: 500px;">
    <i>На этом рисунке показана более общая нейронная сеть. Сеть не полностью подключена, и количество нейронов варьируется от слоя к слою.</i>
</center>

В этом блокноте мы рассмотрим ANN, в которой число нейронов в каждом слое постоянно. Более конкретно, сеть, которую мы будем использовать, называется ResNet и впервые была упомянута в <a href = "https://arxiv.org/abs/1512.03385"> статье </a> Каймина Хэ, Сянъю Чжана, Шаоцина Жэнь и Цзянь Суна. Для простоты число нейронов будет равно размеру входных данных. Мы также ограничим наше внимание полностью связанными нейронными сетями, то есть любой нейрон сети связан со всеми нейронами в следующем слое. Эти упрощения сделаны только для того, чтобы упростить общую структуру для реализации и понимания, но важно подчеркнуть, что выбор более сложных структур может повысить производительность в реальных приложениях. Для таких целей использование хорошо документированных и надежных пакетов Python, таких как PyTorch или TensorFlow, несомненно, проще и лучше, чем пытаться реализовать алгоритмы самостоятельно.

Хотя полносвязная сеть намного проще в реализации, она является более дорогостоящей в вычислительном отношении, чем сеть, которая не полностью связана. Еще одним преимуществом выбора более сложной структуры является то, что она позволяет по-разному обрабатывать подмножества данных. Таким образом, можно в некотором смысле вывести сеть на правильный путь. Кроме того, более сложные структуры допускают более сложные и нелинейные связи между нейронами.

## Обозначения
В этом блокноте будет введено множество переменных. Вот их обзор. 

$$
\begin{equation*}
    \begin{aligned}
        K = & \texttt{ num_layers} && \text{ Количество слоев.} \\
        I = & \texttt{ num_images} && \text{ Количество изображений.}  \\
        d = & \texttt{ dimension} && \text{ Размерность входных данных.} \\
        Y = & \texttt{ Y} && \text{ Все выходные значения, } y \text{, в матрице размера [num layes + 1, num neurons]. } Y[0] \text{ входной слой} \\ 
        W = & \texttt{ weight} &&\text{ Все веса, } w \text{, в матрице размера [num layers, num neurons, num neurons].} \\
        B = & \texttt{ bias_vec} &&\text{ Все смещения, } b \text{, в матрице размера [num layers, num neurons].} \\
        \mu = & \texttt{ mu}  &&\text{ Переменная, соответствующая смещению в выходном слое.}\\
        \omega = & \texttt{ omega}  &&\text{ Переменная, соответствующая весу в выходном слое.}\\
        h = & \texttt{ steplength}  &&\text{ Длина шага.} \\
        Z_i = & \texttt{ Z}  &&\text{ Вывод с последнего слоя, "догадка" сети.} \\
        c_i = & \texttt{ c}  &&\text{ Правильное значение для вывода.}\\
        \mathcal{J} = & \texttt{ cost_function}  &&\text{ Ошибка сети.}\\
        U = & \texttt{ U}  &&\text{ Коллекция всех переменных} W \text{, } b\text{, } \omega \text{ и } \mu \text{.}\\
        \sigma = & \texttt{ sigma}  &&\text{ Сигмовидная функция, обычно используется в качестве функции активации.} \\
        \eta = & \texttt{ eta}  &&\text{ Функция проекции для выходного слоя.} \\
        \sigma ' = & \texttt{ sigma_derivative}  &&\text{ Производная функции активации.} \\
        \eta ' = & \texttt{ eta_derivative}  &&\text{ Производная от проекционной функции.} \\
    \end{aligned}
\end{equation*}
$$

Поскольку то, что мы в конечном итоге получаем, по сути, представляет собой набор множества переменных, которыми мы хотим манипулировать различными способами и в определенных порядках, удобно собирать их в классе. В этом блокноте мы создали три класса, которые мы будем называть $\texttt{Network}$, $\texttt{Param}$ и $\texttt{Gradient_descent}$, и содержание классов будет объяснено по пути. 

## Как работает слой в Сети?

Далее $k$ будет обозначать индекс произвольного слоя, $n$ - произвольный индекс нейрона и $i$ - произвольный индекс входных векторов в сеть. $K$ обозначает общее количество слоев, а $N$ - общее количество узлов в каждом слое.

<center>
    <img src="images/layer_in_nn_2.PNG" alt="A layer in the network" style="width: 250px;">
    <i>На этом рисунке показано, как предыдущий слой является входным для узла в текущем слое. Переменная $k$ является номером текущего слоя и соответствует надстрочному индексу $y$. Индекс $y$ равен $n$, что говорит нам, какому нейрону в слое $y_n$ соответствует элемент.
    </i>
</center>

Каждый слой принимает на вход результат работы предыдущего слоя, за исключением первого слоя, который принимает вход в сеть. В полносвязном слое каждый нейрон принимает выходные данные от каждого нейрона в предыдущем слое в качестве входных данных. В нейроне каждый вход умножается на индивидуальный вес, а затем все они суммируются. Смещение добавляется к сумме, а затем результаты передаются через функцию активации, прежде чем они будут отправлены в качестве выходных данных на следующий слой вместе с выходными данными от каждого другого нейрона в том же слое. Почему нейрон так устроен? Как уже упоминалось в начале, ANN создается для имитации биологической нейронной сети, и в биологической нейронной сети различные стимулы "возбуждают" разные нейроны, и сигнал передается определенным нейронам. В ANN выходное значение нейрона может быть интерпретировано так, как если бы он был "активирован" или нет. Значения, близкие к 1, означают "активацию", в то время как значения, близкие к 0, представляют неактивный нейрон$^2$. Обозначая входы в первый слой $y^{(0)}_n$, веса каждого входа в нейрон, a в первом слое $w_{0,n}^{(0)}$, смещение $b_0^{(0)}$ и функцию активации $f$, выход нейрона $1$ будет равен 

$$
y^{(1)}_0 = y^{(0)}_0 + h f\big( w_{0,0}^{(0)} y^{(0)}_0 + w_{0,1}^{(0)} y^{(0)}_1 + w_{0,2}^{(0)} y^{(0)}_2 + \dots + w_{0,N-1}^{(0)} y^{(0)}_{N-1} + b_0^{(0)} \big) \text{.}
$$ 

$h$ - длина шага и представляет собой число от 0 до 1. Это комбинация выходного значения каждого нейрона в предыдущем слое и их соответствующих весов, которые влияют на сумму в текущем нейроне. Смещение может подтолкнуть значение вверх или вниз, чтобы эффективно создать порог для активации. После преобразования входных данных через скрытые слои они передаются через функцию активации, которая проецирует выходные данные на скаляр между 0 и 1 в случае двоичной классификации. Одним из примеров функции активации является сигмовидная функция: 

$$
\sigma(x) = \frac{1}{1 + e^{-x}}.
$$ 

Чтобы упростить нотацию и вычисления, матрицы и векторы используются для компактного сбора весов и смещений. Пусть входными данными первого слоя являются вектор $y^{(0)}$, $W^{(0)}$ - матрица с весами в первом слое и $b^{(0)}$ - смещения в первом слое. $W$ состоит из векторов с весами от каждого нейрона в слое. Выходные данные первого слоя затем отображаются в матричных обозначениях

$$
\begin{equation*}
    \begin{aligned}
        \begin{bmatrix}
        y^{(1)}_0\\
        y^{(1)}_1\\
        \vdots \\
        y^{(1)}_{N-1}\\
        \end{bmatrix} =
        \begin{bmatrix}
        y^{(0)}_0\\
        y^{(0)}_1\\
        \vdots \\
        y^{(0)}_{N-1}\\
        \end{bmatrix}
        +
        h \sigma \left(
        \begin{bmatrix}
        w^{(0)}_{0,0} & w^{(0)}_{0,1} & \dots & w^{(0)}_{0,N-1} \\
        w^{(0)}_{1,0} & w^{(0)}_{1,1} & \dots & w^{(0)}_{1,N-1} \\
        &\vdots \\ 
        w^{(0)}_{N-1,0} & w^{(0)}_{N-1,1} & \dots & w^{(0)}_{N-1,N-1} \\
        \end{bmatrix}
        \begin{bmatrix}
        y^{(0)}_0\\
        y^{(0)}_1\\
        \vdots \\
        y^{(0)}_{N-1}\\
        \end{bmatrix}
        +
        \begin{bmatrix}
        b^{(0)}_0\\
        b^{(0)}_1\\
        \vdots \\
        b^{(0)}_{N-1}\\
        \end{bmatrix}
        \right)
        \text{,}
    \end{aligned}
\end{equation*}
$$
или написано более компактно

$$
y^{(1)} = y^{(0)} + h \sigma \left(W^{(0)}y^{(0)} + b^{(0)}\right) \text{,}
$$

где сигмоидная функция$^3$, $\sigma$, применяется поэлементно. В полносвязной нейронной сети каждый нейрон в одном слое соединен с каждым нейроном в следующем слое, и приведенное выше матричное уравнение дает выход каждого скрытого слоя в сети. Для вычислительных целей удобно отправлять каждый входной вектор через преобразование одновременно. В нашей нотации это равносильно 

\begin{equation}
    \mathbf{Y}_{k} = \mathbf{Y}_{k-1} + h \sigma \left( W_{k-1} \mathbf{Y}_{k-1} + b_{k-1}\right).
\end{equation}

## Функции активации
Функции активации - это то, что отличает нейронную сеть от линейной регрессии. Без обработки выходных данных каждого слоя с помощью нелинейной функции активации всегда можно было бы создать один слой, равный сумме любого другого набора слоев того же размера, и глубина сети была бы незначительной. С помощью функций активации сеть может устанавливать нелинейные соединения между входом и выходом. Другой аспект функции активации заключается в том, что становится более ясно, активен или неактивен нейрон. Можно было бы подумать, что лучшим способом показать это будет двоичная функция активации, которая выводит 1, если она активна, и 0, но тогда градиент функции будет плохо определен. Станет ясно, что это очень прискорбно, когда мы обсудим тренировку сети, где градиент функции активации играет решающую роль. Существует множество вариантов функций активации, некоторые из наиболее известных: сигмоидная функция, гиперболический тангенс и функция ReLU.

### Сигмоида
Сигмовидная функция является хорошо известной функцией активации, и она принимает значения от 0 до 1. Это непрерывная функция, которая упрощает вычисления градиента. Одним из недостатков этой функции является то, что называется "исчезающими градиентами". то есть, когда абсолютное значение входных данных принимает большое значение, градиент сигмоидной функции становится очень маленьким. Как мы увидим позже, градиент функции активации является важной частью обучения сети, и исчезающие градиенты заставят сеть учиться очень медленно [[2]](#activation_functions). Эта функция также может использоваться в выходном слое.

In [ ]:
# Packages:
import pickle  # Сериализация объектов Python.

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns  # Библиотека статистической графики.
from IPython.display import Image
from matplotlib import rc
from tqdm import tqdm  # Модные измерители прогресса.

In [ ]:
# Setting common plotting parameters
fontsize = 22
newparams = {
    "axes.titlesize": fontsize,
    "axes.labelsize": fontsize,
    "lines.linewidth": 2,
    "lines.markersize": 7,
    "figure.figsize": (13, 7),
    "ytick.labelsize": fontsize,
    "xtick.labelsize": fontsize,
    "legend.fontsize": fontsize,
    "legend.handlelength": 1.5,
    "figure.titlesize": fontsize,
    "figure.dpi": 400,
    "text.usetex": True,
    "font.family": "sans-serif",
}
plt.rcParams.update(newparams)

In [ ]:
def sigmoid(x):
    return np.exp(x)/(np.exp(x) + 1)

def sigmoid_derivative(x):
    return 1/np.square(np.exp(x/2)+np.exp(-x/2))

x = np.linspace(-10,10,200)
plt.plot(x, sigmoid(x),label=r"$\sigma (x) = \frac{\exp{x}}{\exp{x} +1}$")
plt.plot(x, sigmoid_derivative(x),label=r"$\sigma '(x) = \left(\frac{1}{ \exp{\left(\frac{x}{2}\right)} + \exp{\left(-\frac{x}{2}\right)}}\right)^2$")
plt.title("Sigmoid function")

plt.tight_layout()
plt.legend()

plt.show()

### Гибридный тангенс

Форма гиперболического тангенса аналогична сигмоидной функции, но она принимает значения от $-1$ до $1$. Согласно [[3]](#ann_training), центрирование значений вокруг 0 облегчит обучение сети, и это часто дает лучшие результаты, чем сигмоидальная функция.

In [ ]:
def tanh(x):
    return (np.exp(2*x)-1)/(np.exp(2*x)+1)

def tanh_derivative(x):
    return 4/np.square(np.exp(x)+np.exp(-x))

x = np.linspace(-10,10,200)
plt.plot(x, tanh(x),label=r"$\eta (x) = \tanh{x}$")
plt.plot(x, tanh_derivative(x),label=r"$\eta '(x) = \frac{1}{\cosh^2{x}}$")
plt.title("Hyperbolic tangent")

plt.tight_layout()
plt.legend()

plt.show()

### ReLU

ReLU - это сокращение от выпрямленной линейной единицы и она менее затратна в вычислительном отношении, чем сигмоидные и гиперболические касательные функции. Она возвращает 0, если входное значение отрицательное, и само значение, если входное значение положительное. Эта функция не имеет верхнего предела для выходных значений, но она четко показывает, когда нейрон неактивен. Существуют версии ReLU, в которых не все отрицательные значения становятся нулевыми, например, дырявый ReLU, который имеет небольшой линейный наклон на отрицательной стороне. Для некоторых проблем полезно выводить ноль для отрицательных значений, потому что это ясно показывает, что нейрон неактивен, но некоторые нейроны могут в конечном итоге выводить только 0, и нейрон не вносит свой вклад в сеть[[4]](#ReLU).

In [ ]:
def ReLU(x):
    return np.maximum(0,x)

def leakyReLU(x, a):
    return np.maximum(x*a, x)

def ReLU_derivative(x):
    return np.heaviside(x,0)

def leakyReLU_derivative(x, a):
    return a + np.heaviside(x,0) * (1-a)

x = np.linspace(-10,10,200)

fig, axs = plt.subplots(1, 2)
axs[0].plot(x, ReLU(x), label = "ReLU")
axs[0].plot(x, ReLU_derivative(x), label = "Derivative of ReLU", ls = "-")
axs[1].plot(x, leakyReLU(x,0.1), label = "leaky ReLU")
axs[1].plot(x, leakyReLU_derivative(x, 0.1), label = "Derivative of leaky ReLU", ls = "-")
axs[0].legend()
axs[1].legend()
axs[0].set_title('ReLU')
axs[1].set_title('Leaky ReLU')

plt.tight_layout()
plt.show()

### Выходной слой

Наиболее важной частью выходного слоя является уменьшение размера входных данных. В этом блокноте задача представляет собой задачу двоичной классификации, а желаемый результат - число от 0 до 1. Для других типов задач выход может быть, например, вектором или действительными числами. В то время как большая часть сети может оставаться неизменной для других проблем, важно адаптировать выходной уровень, чтобы получить желаемые размеры и значения. Типичным выходным слоем является 

$$
Z = \eta (\mathbf{Y}_K^T \omega + \mu \mathbf{1}).
$$

Здесь $\omega$ является вектором и выполняет ту же работу, что и веса в скрытом слое. $\mu$ является одномерным вектором (т. е. скаляром) и выполняет работу смещения, а $\mathbf{1}$ обозначает вектор той же размерности, что и $\omega$, содержащий только единицы. $\eta$ - это функция масштабирования. Для целей двоичной классификации $\eta$ обычно выводит десятичное число от 0 до 1, а сигмоидная функция является одной из многих функций, которые можно использовать. Если желательно вывести вектор, $\omega$ будет матрицей, $\mu$ будет вектором, а $\eta$ будет работать поэлементно. Вывод $Z$ является предположением сети и реализуется следующим образом:

In [ ]:
# Метод, принадлежащий сетевому классу.
# self = Network
# U = Параметр, принадлежащий классу Param

def projection(self):
    self.Z = self.eta(Y[-1].T@self.U.omega + self.U.mu)

Во время обучения нейронной сети это значение сравнивается с истинным значением, обозначаемым $c$, для расчета ошибки сети. То есть насколько далека была догадка. При запуске все параметры по существу свободны, поэтому мы ожидаем, что предположение будет близким к случайному. Чтобы улучшить догадку, идея состоит в том, чтобы изменить веса, смещения, $\omega$ и $\mu$ таким образом, чтобы минимизировать ошибку.

## Как обучается сеть?

Если сеть уже обучена, она может принять решение, "съев" входные данные, обработав их через скрытые слои и, в конечном счете, через выходной слой, из которого она принимает решение. Прежде чем это станет возможным, сеть должна быть обучена. Обучение нейронной сети означает, что каждый вес и смещение настраиваются на проблему, которую вы хотите решить, чтобы сделать решения как можно более точными. Это делается в основном в два этапа: прямое распространение и обратное распространение.

### Прямое распространение

При прямом распространении мы отправляем входные данные и позволяем сети сделать предположение. В начале обучения эти предположения будут в основном случайными, так как веса и смещения не адаптированы к набору данных. Предположение делается путем простой отправки входных данных через все слои.

In [ ]:
# Метод, принадлежащий сетевому классу.
# self = Network
# U = Параметр, принадлежащий классу Param

def forward_prop(self):
    for i in range(self.num_layers):
        self.Y[i+1,:,:] = self.steplength*self.sigma(self.U.weight[i,:,:]@self.Y[i,:,:] + self.U.bias_vec[i])

### Обратное распространение

Когда предположение было сделано, ошибка между предположением и истинным значением измеряется с помощью функции затрат. Целью следующего шага будет использование информации об ошибке для улучшения последующих догадок. Нормальный выбор для функции затрат, $\mathcal{J}$, - это квадратное отклонение догадок и истинных значений

$$
\mathcal{J} = \frac{1}{2} \sum_{i=1}^{I} \vert Z_i - c_i \vert^2 = \frac{1}{2} \| \mathbf{Z} - \mathbf{c} \|^2.
$$

Когда речь идет о входных данных $\mathbf{Y}_k$, как указано, функция ошибок является просто функцией параметров сети. Для простоты мы собираем все параметры в одну переменную под названием $\mathbf{U} = [(W_k, b_k)_{k=0}^{K-1},\omega,\mu]$. Мы хотим найти значения параметров $\mathbf{U}$, которые минимизируют $\mathcal{J}$. Поскольку $\mathcal{J}$ быстрее всего уменьшается локально в направлении $-\nabla J (\mathbf{U})$, мы обновляем наши параметры в соответствии с алгоритмом градиентного спуска, где одним из простейших алгоритмов является итерация 

$$
\mathbf{U}^{(j+1)} = \mathbf{U}^{(j)} - \tau \nabla J (\mathbf{U}^{(j)}) \text{,}
$$

где $\tau$ известен как параметр обучения. При обратном распространении вычисляется градиент функции затрат по отношению ко всем переменным и используется для обновления весов, чтобы минимизировать стоимость сети. 

In [ ]:
# Method belonging to network
# self = Network

def calculate_cost(self):
    self.cost = 1/2*np.sum(np.square(self.Z-self.c))

### Расчет градиентов

Чтобы найти параметры, которые минимизируют стоимость $\mathcal{J}$, мы используем итерационную схему, включающую локальный градиент стоимости $\nabla \mathcal{J}(\mathbf{U}^{(j)})$. Далее мы выведем простейшие компоненты градиента и обратимся к приложению для вывода более сложных. Возможно, самым простым является тот, который относится к скалярному $\mu$, используемому в выходном слое.

\begin{align}
    \frac{\partial \mathcal{J}}{\partial \mu} = \sum_{i=1}^{I} \frac{\partial \mathcal{J}}{\partial Z_i} \frac{\partial Z_i}{\partial \mu} &= \sum_{i=1}^{I} [\eta'(\mathbf{Y}_K^T \omega + \mu \mathbf{1})]_i (Z_i-c_i) \\ &= \eta'(\mathbf{Y}_K^T \omega + \mu \mathbf{1})^T (\mathbf{Z}- \mathbf{c}).
\end{align}

Вычисление градиента относительно $\mu$ выполняется следующим образом.

In [ ]:
# Method belonging to Parameters
# self = Param

def gradient_mu(self,network):
    first_factor = network.eta_derivative(network.Y[self.num_layers,:,:].T @ self.omega + self.mu * network.one).T
    second_factor = network.Z - network.c

    return first_factor @ second_factor 

Аналогично получим

\begin{align}
    \frac{\partial \mathcal{J}}{\partial \omega} &= \sum_{i=1}^{I}  \frac{\partial \mathcal{J}}{\partial Z_i} \frac{\partial Z_i}{\partial \omega} = \sum_{i=1}^{I} \sum_{j=1}^{d} (Z_i - c_i) [ \eta'(\mathbf{Y}_K^T \omega + \mu \mathbf{1})]_i \mathbf{Y}^T_{K,ij} \\
    &= \mathbf{Y}_K^T \left( \left( \mathbf{Z} - \mathbf{c} \right) \odot \eta'(\mathbf{Y}_K^T \omega + \mu \mathbf{1}) \right),
\end{align}

где мы ввели произведение Адамара (поэлементное) $\odot$, определяемое $(A \odot B)_{ij} = A_{ij} \cdot B_{ij}$. В numpy мы можем вычислить произведение Адамара двух массивов, $\texttt{X}$ и $\texttt{Y}$ (с одинаковой формой), используя `np.multiply(X,Y)` или просто `X * Y`, где обычное умножение матрицы выполняется с помощью оператора `@` или `np.dot(,)`.

In [ ]:
# Method belonging to Parameters
# self = Param

def gradient_omega(self,network):
    
    first_factor = network.Y[self.num_layers,:,:]
    second_factor = np.multiply((network.Z - network.c),network.eta_derivative(first_factor.T @ self.omega + self.mu * self.one))

    return first_factor @ second_factor

Вычисление градиента по отношению к смещению и весам немного запутаннее, поэтому мы представляем результаты только здесь и приводим подробности в приложении. Оказывается полезным вычислить градиент по отношению к $\mathbf{Y}_k$, чтобы получить эти градиенты. Мы обозначим $\mathbf{P}$ и представим следующие тождества, соединяющие $\mathbf{P}$ последнего слоя с предыдущими. 

\begin{equation}\label{eq:P_K}
\mathbf{P}_K = \frac{\partial \mathcal{J}}{\partial \mathbf{Y}_K} = \omega \otimes \left[(\mathbf{Z} - \mathbf{c}) \odot \eta'\left( \mathbf{Y}_{K} ^{T} \omega + \mu \mathbf{1}\right) \right]^T 
\end{equation}

\begin{equation}\label{eq:P_k-1}
\mathbf{P}_{k-1} = \frac{\partial \mathcal{J}}{\partial \mathbf{Y}_{k-1}} = \mathbf{P}_{k} + h W_{k-1}^{T} \cdot \left[ \sigma' \left(W_{k-1} \mathbf{Y}_{k-1} + b_{k-1} \right) \odot \mathbf{P}_{k}\right] 
\end{equation}

Здесь $\otimes$ обозначает внешнее произведение. Используя их, мы можем выразить градиент по отношению к весам и смещениям следующим образом 

\begin{equation}
    \frac{\partial \mathcal{J}}{\partial W_k} = h \left( \mathbf{P}_{k+1} \odot \sigma' \left( W_k \mathbf{Y}_k + b_k \right) \right) \cdot \mathbf{Y}_{k}^{T},
\end{equation}

и  

\begin{equation}
    \frac{\partial \mathcal{J}}{\partial b_k} = h \left( \mathbf{P}_{k+1} \odot \sigma' \left( W_k \mathbf{Y}_k + b_k \right) \right) \cdot \mathbf{1}.
\end{equation}

Обратите внимание, что для их вычисления требуется сначала вычислить $\mathbf{P}_K$ на основе $\mathbf{Y}_K$, который получается прямым распространением. После этого можно вычислить $\mathbf{P}_{k}$ для $k<K$, и все компоненты $\mathbf{P}$ необходимы для вычисления градиентов относительно всех весов и смещений.

In [ ]:
# Method belonging to Parameters
# self = Param

def calculate_P_K(self,network):

    first_factor  = network.Z - network.c
    second_factor = network.eta_derivative((network.Y[self.num_layers,:,:]).T @ self.omega + self.mu * network.one) 
    third_factor = np.multiply(first_factor,second_factor)

    self.P[self.num_layers,:,:] =  np.outer(self.omega, third_factor.T)
        
def calculate_P(self,network):
    for k in range(self.num_layers,0,-1):
        first_factor  = network.steplength * self.weight[k-1,:,:].T 
        second_factor = np.multiply(network.sigma_derivative(self.weight[k-1,:,:] @ network.Y[k-1,:,:] + self.bias_vec[k-1]),self.P[k,:,:])

        self.P[k-1,:,:]  = self.P[k,:,:] + first_factor @ second_factor

In [ ]:
# Method belonging to Parameters
# self = Param

def gradient_weight(self,network,index):

    first_factor  = network.steplength * np.multiply(self.P[index+1,:,:],network.sigma_derivative(self.weight[index,:,:] @ network.Y[index,:,:] + self.bias_vec[index]))
    second_factor = network.Y[index,:,:].T
    
    return first_factor @ second_factor 

def gradient_bias_vec(self,network,index):

    first_factor  = network.steplength * np.multiply(self.P[index+1,:,:],network.sigma_derivative(self.weight[index,:,:] @ network.Y[index,:,:] + self.bias_vec[index]))
    second_factor  = network.one

    return np.reshape(first_factor @ second_factor,(self.dimension,1))

### Инициализация параметров

При инициализации параметров $\mathbf{U}$ в сети можно наивно думать, что простой выбор - инициализировать все параметры  нулями с помощью `np.zeros()`, но это не очень хороший вариант. Если все значения весов равны нулю, градиент будет равен единице для всех весов, и градиент не будет меняться для всех весов, и сеть будет работать как линейная модель [[5]](#initialization_of_weights). Существует много способов улучшить модель, инициализируя параметры таким образом, чтобы улучшить обучение, но мы сохраним ее простой и инициализируем параметры с использованием нормального распределения. В следующей ячейке мы создали класс для всех параметров в $\mathbf{U}$ и их градиентов.

In [ ]:
class Param(object):
    """Параметры нейронной сети.
    
    Инициализирует параметры случайными числами.

    Parameters
    ----------

    K : int
        number of layers
    d : int
        dimension of input 'images'
    I : int
        number of input input 'images'
    
    Attributes
    ----------
    
    num_layers : int
        number of layers in total
    dimension  : int
        dimension of input 'image'
    num_images : int
        number of input images 
        
    mu         : float
        mu in projection/output layer
    omega      : np.array
        omega in projection/output layer. shape: dimension x 1
    weight     : np.array
        weights. shape : num_layers x dimension x dimension
    bias_vec   : np.array
        bias. shape: num_layers x dimension x num_images   
    P          : np.array
        P-matrix. shape: num_layers + 1 x dimension x num_images
    """
    
    def __init__(self,K,d,I):
        self.num_layers = K
        self.dimension  = d
        self.num_images = I
        
        self.mu         =  np.random.normal()
        self.omega      =  np.random.randn(self.dimension,1)
        self.weight     =  np.random.randn(self.num_layers,self.dimension,self.dimension)
        self.bias_vec   =  np.random.randn(self.num_layers,self.dimension,1)
        
        self.P          =  np.zeros((self.num_layers+1,self.dimension,self.num_images))
        
    def gradient_mu(self,network):
        """Вычисляет градиент относительно mu
        
        Parameters
        ----------
            network : Network
                The network of which this instance is a member
        Returns
        -------
            _ : float
                The gradient with respect to mu
        """
        
        first_factor = network.eta_derivative(network.Y[self.num_layers,:,:].T @ self.omega + 
                                              self.mu * network.one).T
        second_factor = network.Z - network.c
    
        return first_factor @ second_factor 
    
    def gradient_omega(self,network):
        """Вычисляет градиент по отношению к омеге
        
        Parameters
        ----------
            network : Network
                The network of which this instance is a member
        Returns
        -------
            _ : np.array
                The gradient with respect to omega
        """
    
        first_factor = network.Y[self.num_layers,:,:]
        second_factor = np.multiply((network.Z - network.c),
                                    network.eta_derivative(first_factor.T @ self.omega + self.mu * network.one)) 

        return first_factor @ second_factor
    
    def calculate_P_K(self,network):
        """Вычисляет P_K
        
        Parameters
        ----------
            network : Network
                The network of which this instance is a member
        """
    
        first_factor  = network.Z - network.c
        second_factor = network.eta_derivative((network.Y[self.num_layers,:,:]).T @ self.omega + 
                                               self.mu * network.one) 
        third_factor = np.multiply(first_factor,second_factor)
        
        self.P[self.num_layers,:,:] = np.outer(self.omega, third_factor.T)
        
    def calculate_P(self,network):
        """Вычисляет P 
        
        Parameters
        ----------
            network : Network
                The network of which this instance is a member
                
        """
        
        for k in range(self.num_layers,0,-1):
            first_factor  = network.steplength * self.weight[k-1,:,:].T 
            second_factor = np.multiply(network.sigma_derivative(self.weight[k-1,:,:] @ network.Y[k-1,:,:] 
                                                                 + self.bias_vec[k-1]),self.P[k,:,:])

            self.P[k-1,:,:]  =  self.P[k,:,:] + first_factor @ second_factor
            
    def gradient_weight(self,network,index):
        """Вычисляет градиент по отношению к весу
        
        Parameters
        ----------
            network : Network
                The network of which this instance is a member
        Returns
        -------
            _ : np.array
                The gradient with respect to the weight
        """
  
        first_factor  = network.steplength * np.multiply(self.P[index+1,:,:],
                        network.sigma_derivative(self.weight[index,:,:] @ network.Y[index,:,:] +
                                                  self.bias_vec[index]))
        second_factor = network.Y[index,:,:].T
        return first_factor @ second_factor 
        
    def gradient_bias_vec(self,network,index):
        """Вычисляет градиент по отношению к смещению
        
        Parameters
        ----------
            network : Network
                The network of which this instance is a member
        Returns
        -------
            _ : np.array
                The gradient with respect to the bias
        """

        first_factor  = network.steplength * np.multiply(self.P[index+1,:,:],
                        network.sigma_derivative(self.weight[index,:,:] @ network.Y[index,:,:] +
                                                 self.bias_vec[index]))
        second_factor  = network.one
        
        return np.reshape(first_factor @ second_factor,(self.dimension,1))
    

### Алгоритм обучения

Прежде чем объяснить методы градиентного спуска, мы дадим краткое описание процесса обучения:

for $i$ in range(num_iterations): <br>
$\hspace{1cm}$ for $k$ in range($K$): <br>
$\hspace{2cm}$ Вычислить $Y_k$ <br>
$\hspace{1cm}$ Вычислить $P_K$ <br>
$\hspace{1cm}$ Вычислить градиент $\omega$ и $\mu$ <br>
$\hspace{1cm}$ for $k$ in range(K-1, 1, -1): <br>
$\hspace{2cm}$ Вычислить $P_{k-1}$ <br>
$\hspace{1cm}$ for $k$ in range(K-1): <br>
$\hspace{2cm}$ Вычислить градиент $W_k$ и $b_k$ <br>
$\hspace{1cm}$ Обновление $\mathbf{U}$ по методу градиентного спуска <br>

В этом блокноте мы решили создать класс для каждого метода градиентного спуска. Чтобы гарантировать, что различные классы для градиентных спусков работают в сети, мы определили три функции, которые должны содержать все классы градиентного спуска. Это `update_first()`, `update_second()` и `update_params()`. `update_first()` вычисляет градиенты $\mu$ и $\omega$, `update_second()` вычисляет градиенты $W$ и $b$, а `update_params()` обновляет все значения $U$ в соответствии с методом градиентного спуска. Реализация этих функций будет показана в следующем разделе. Реализация алгоритма обучения показана ниже. Строки кода, за которыми следует "##", не способствуют трансированию, но используются для построения графика для проверки сети. 

In [ ]:
# Method beloning to the Network class
# self = Network


def train(self,h = 0.1,tau = 0.01):

    self.cost_per_iter = np.zeros(self.iterations-1)
    self.validation_cost_per_iter = np.zeros(self.iterations-1) ##
    self.steplength = h
    self.tau = tau

    for i in tqdm(range(self.iterations)): #tqdm creates a progressbar

        self.forward_prop(self.Y)

        self.Z = self.projection(self.Y)
        self.U.calculate_P_K(self)

        self.gradient_descent.update_first(self)
        self.U.calculate_P(self)

        self.gradient_descent.update_second(self)

        self.Z = self.projection(self.Y)

        self.gradient_descent.update_params(self,i)

        self.Z = self.projection(self.Y)

        if i != 0:
            self.cost_per_iter[i - 1] = self.cost_function()
            pred = self.predict(self.validation_data, integers = False) ##
            self.validation_cost_per_iter[i-1] = 1/2*(np.linalg.norm(pred-self.validation_labels))**2 ##
                

### Оптимизация

Как уже упоминалось, мы хотим найти значения $\mathbf{U}$, которые минимизируют $\mathcal{J}$. Есть много способов сделать это, и мы представим два метода: простой градиентный спуск и градиентный спуск Адама.

#### Обычная ваниль

Простой ванильный градиентный спуск является одним из простейших алгоритмов градиентного спуска и обновляет $\mathbf{U}$ с итерацией 

$$
\mathbf{U}^{(j+1)} = \mathbf{U}^{(j)} - \tau \nabla J (\mathbf{U}^{(j)}) \text{.}
$$ 

In [ ]:
class GradientDescent:
    """Виртуальный класс для методов градиентного спуска для сети. 
    Все методы градиентного спуска должны иметь эту форму, чтобы быть совместимыми с классом сети
    
    Parameters
    -----------
    network : Network
        Сеть, объектом которой является данный экземпляр 
    """
    
    def __init__(self, network):
        self.gradient_mu = np.zeros(np.shape(network.U.mu))
        self.gradient_omega = np.zeros(np.shape(network.U.omega))
        self.gradient_bias_vec = np.zeros(np.shape(network.U.bias_vec))
        self.gradient_weight = np.zeros(np.shape(network.U.weight))
    
    def update_first(self, network):
        """Обновляет градиенты wrt mu и omega
        
        Parameters
        ----------
        network : Network
            Сеть, объектом которой является данный экземпляр 
        """
        self.gradient_mu = network.U.gradient_mu(network)
        self.gradient_omega = network.U.gradient_omega(network)
        
    def update_second(self, network):
        """Обновление градиентов весов wrt и смещений
        
        Parameters
        ----------
        network : Network
            Сеть, объектом которой является данный экземпляр 
        """
        for k in range(network.num_layers):
            self.gradient_weight[k] = network.U.gradient_weight(network, k)
            self.gradient_bias_vec[k] = network.U.gradient_bias_vec(network, k)
            
    def update_params(self, network, j):
        """Обновляет параметры сети после вычисления градиентов 
        
        Parameters
        ----------
        network : Network
            Сеть, объектом которой является данный экземпляр 
        j    : int
            iteration of training.
            
        """
        raise NotImplementedError

        
class Plain_vanilla(GradientDescent):
    """Класс простой ванильный градиентный спуск. 
    Наследует класс формы GradientDescent
    
    Parameters
    -----------
    network : Network
        Сеть, объектом которой является данный экземпляр 
    tau : float
        длина шага, используемая в алгоритме простой ванили
    """
    def __init__(self, network, tau=0.01):
        GradientDescent.__init__(self, network)
        self.tau = tau    
            
    def update_params(self, network, j):
        """Обновляет параметры сети после вычисления градиентов 
        
        Parameters
        ----------
        network : Network
            Сеть, объектом которой является данный экземпляр 
        iter    : int
            итерация обучения. Здесь это не имеет значения, но важно для адамовского спуска.
        """
        network.U.mu = network.U.mu - self.tau * self.gradient_mu
        network.U.omega = network.U.omega - self.tau * self.gradient_omega
        network.U.weight = network.U.weight - self.tau * self.gradient_weight    
        network.U.bias_vec = network.U.bias_vec - self.tau * self.gradient_bias_vec

#### Адамов спуск

Алгоритм спуска Адама не так прост, как простой градиентный. Одно из различий между этими двумя алгоритмами заключается в том, что обычная ваниль использует одну и ту же длину шага на протяжении всего процесса обучения, в то время как Адам адаптирует длину шага к градиенту. Мы не будем углубляться в то, как работает Адам, но вы можете прочитать об этом подробнее <a href = "https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/" target =_blank >здесь</a>.

Это алгоритм для метода градиентного спуска Адама:


$v_0 = 0$, $m_0 = 0$ <br>
for $j = 1$,$2$, $\dots$ <br>
    $\hspace{1cm} g_j = \nabla_{\mathbf{U}} \mathcal{J}(\mathbf{U}^{(j)})$ <br>
    $\hspace{1cm} m_j = \beta_1 m_{j-1} + (1-\beta_1) g_j$ <br>
    $\hspace{1cm} v_j = \beta_2 v_{j-1} + (1-\beta_2)(g_j \odot g_j)$ <br>
    $\hspace{1cm} \hat{m}_j = \frac{m_j}{1-\beta_1^j}$ <br>
    $\hspace{1cm} \hat{v}_j = \frac{v_j}{1-\beta_2^j}$ <br>
    $\hspace{1cm} \mathbf{U}^{(j+1)} = \mathbf{U}^{(j)} - \alpha \frac{\hat{m}_j}{\sqrt{\hat{v}_j} + \epsilon}$


где $\beta_1$, $\beta_2$, $\alpha$ и $\epsilon$ - параметры, которые можно изменить для оптимизации производительности алгоритма.
$\odot$ по-прежнему является произведением Адамара.

In [ ]:
class Adam(GradientDescent):
    """Класс градиентного спуска Адама. 
    Наследует класс формы GradientDescent
    
    Parameters
    -----------
    network : Network
        Сеть, объектом которой является данный экземпляр 
    tau : float
        длина шага, используемая в алгоритме Adam
    """
    def __init__(self,network,tau = 0.01):
        GradientDescent.__init__(self, network)
        self.tau = tau
        
    def update_params(self,network,j):
        """Обновляет параметры сети после вычисления градиентов 
        
        Parameters
        ----------
        network : Network
            Сеть, объектом которой является данный экземпляр 
        j    : int
            итерация обучения   
        """ 
        
        beta_1  = 0.9
        beta_2  = 0.999
        alpha   = self.tau
        epsilon = 1e-8

        if j == 0:
            self.m = 0
            self.v = 0
        else:
            g_j = np.asarray([self.gradient_mu,self.gradient_omega,
                              self.gradient_weight,self.gradient_bias_vec], dtype=object)

            self.m = beta_1 * self.m + (1-beta_1) * g_j
            self.v = beta_2 * self.v + (1-beta_2) * np.multiply(g_j,g_j)

            m_hat = self.m /(1-beta_1**j) 
            v_hat = self.v /(1-beta_2**j)

            network.U.mu = network.U.mu - alpha * m_hat[0] /(np.sqrt(v_hat[0]) + epsilon)
            network.U.omega = network.U.omega - alpha * m_hat[1] /(np.sqrt(v_hat[1]) + epsilon)
            network.U.weight = network.U.weight - alpha * m_hat[2] /(np.sqrt(v_hat[2]) + epsilon)    
            network.U.bias_vec = network.U.bias_vec - alpha * m_hat[3] /(np.sqrt(v_hat[3]) + epsilon)

## Настройка сети

Прежде чем мы соберем весь класс сети, представим некоторые полезные функции при работе с машинным обучением. 

### Сохранение и загрузка моделей

Нейронная сеть, созданная здесь, довольно проста, и она обучается относительно быстро. Для более глубоких сетей, которые применяются к более сложным задачам, обычно требуется больше работать с поиском хороших параметров, например, количества слоев, количества итераций и размера различных слоев. Время, необходимое для обучения модели, также может быть значительно больше, чем для этой сети. Вот некоторые из причин, по которым полезно иметь возможность сохранять и загружать модели во время разработки сети. Когда вы сможете сохранять и загружать свои модели, вам будет легче сравнивать модели, не тренируя их каждый раз. Для удобства мы также включаем методы для этого с помощью простой сети. 
Мы решили использовать словари Python для сохранения моделей. Таким образом, можно сохранить все функции формы в массивах numpy в одном документе. Мы также сохраняем словарь в двоичном файле с помощью функции `pickle.dump`. 

In [ ]:
# Методы, относящиеся к классу сети.
# self = Network

def save_model(self, filename):
    """Сохраняет обученную модель. Сохраняет все значения, необходимые для прогнозирования с помощью модели. 

    Parameters
    ----------
    filename : string
        Имя двоичного файла для создания модели.
    """
    # Создание вложенных словарей для структурирования данных
    parameters = {}
    parameters['weight'] = self.U.weight
    parameters['bias_vec'] = self.U.bias_vec
    parameters['mu'] = self.U.mu
    parameters['omega'] = self.U.omega

    dimensions = {}
    dimensions['K'] = self.num_layers
    dimensions['d'] = self.dimension
    dimensions['I'] = self.num_images
    dimensions['iterations'] = self.iterations 

    functions = {}
    functions['sigma'] = self.sigma
    functions['sigma_derivative'] = self.sigma_derivative
    functions['eta'] = self.eta
    functions['eta_derivative'] = self.eta_derivative

    # Соберет все словари в одном
    network_dict = {}
    network_dict['parameters'] = parameters
    network_dict['dimensions'] = dimensions
    network_dict['functions'] = functions

    # Сохранит словарь как бинарный файл
    # 'w' для записи, 'b' открыть как двоичный файл (по умолчанию используется текстовый файл)
    with open(filename, 'wb') as outfile:
        pickle.dump(network_dict, outfile, pickle.HIGHEST_PROTOCOL)

def load_model(self, filename):
    """Загружает обученную модель. Устанавливает все значения, необходимые для прогнозирования с помощью модели. 

    Parameters
    ----------

    filename : string
        Имя двоичного файла для загрузки формы модели. Файл должен содержать словарь.
    """
    # Открыть файл как двоичный файл
    # "r" для чтения, "b", чтобы открыть файл в виде двоичного файла (по умолчанию это текстовый файл)
    file_to_read = open(filename, "rb")

    # Считывает двоичный файл с помощью pickle
    network_dict = pickle.load(file_to_read)

    self.num_layers = network_dict['dimensions']['K']
    self.dimension = network_dict['dimensions']['d']
    self.num_images = network_dict['dimensions']['I']
    self.iterations = network_dict['dimensions']['iterations']

    # Initalize U with random values
    self.U = Param(self.num_layers, self.dimension, self.num_images)

    # Set the right values for U
    self.U.weight = network_dict['parameters']['weight']
    self.U.bias_vec = network_dict['parameters']['bias_vec']
    self.U.mu = network_dict['parameters']['mu']
    self.U.omega = network_dict['parameters']['omega']

    self.sigma = network_dict['functions']['sigma']
    self.sigma_derivative = network_dict['functions']['sigma_derivative']
    self.eta = network_dict['functions']['eta']
    self.eta_derivative = network_dict['functions']['eta_derivative']

### Структурирование сети

Как видно из этого блокнота, нейронная сеть состоит из множества переменных и функций, и для того, чтобы все было аккуратно и читабельно, важно структурировать код. Мы решили разделить реализацию параметров и сети на два отдельных класса, а также сохранить отдельный класс для метода градиентного спуска. Обратите внимание, что, конечно, есть много способов сделать это, которые будут работать одинаково хорошо. В следующей ячейке кода мы собрали класс `Network`. Большинство функций и переменных уже были введены, но функции, предназначенные для тестирования сети, будут объяснены в следующих разделах.

In [ ]:
class Network():
    def __init__(self, K=None, d=None, I=None, 
                 num_iterations = None, 
                 activation_functions_list = None, 
                 gradient_descent_method = None, 
                 gradient_descent_input = [], 
                 filename = '', data = None,
                 rstate = 42
                ):
        """Инициализирует сеть. Типы инициализации: из файла с заданными переменными или из набора данных.
        -------
        Из файла с установленными переменными:
        Input
        ------
        filename : string
            имя файла со всеми данными, необходимыми для прогнозирования. Должен:
                - Быть двоичным файлом
                - Содержать словарь Python в формате, указанном в Network.save_model()
        -------
        Из набора данных
        Input
        -----
        K : int
            Количество слоев
        d : int
            Количество нейронов в слое
        I : int
            Количество входов
        activation_functions_list : список функций
            [функция активации, производная функции активации, функция активации выходного слоя, 
            производная функции активации выходного слоя]
        gradient_descent_method : GradientDescent
            Класс для градиентного спуска 
        gradient_descent_input : list
            Список дополнительных документов к gradient_descent_methon. Default = []
        rstate : int
            Random state for numpy. Чтобы сделать выходные данные воспроизводимыми
            
            
        Attributes
        ----------
            U                : Param
                Параметры сети.
            num_layers       : int
                Общее количество слоев
            dimension        : int
                Размер входного "изображения"
            num_images       : int
                Количество входных изображений 
            gradient_descent : GradientDescent
                Метод градиентного спуска
            one              : np.array
                Array of ones. shape: num_images x 1
            Y                : np.array
                Матрица, содержащая изображение, преобразованное через K слоев. 
                shape: num_layers + 1 x dimension x num_images
            Z                : np.array
                Массив, содержащий проекцию последнего слоя в каждой итерации. размер: dimension x 1
            c                : np.array
                Истинные метки изображений. размер: dimension x 1
            sigma            : function
                Сигмовидная функция - функция активации для использования при преобразовании через скрытые слои
            sigma_derivative : function
                Производная сигмовидной функции
            eta              : function
                Функция активации для проецирования последнего слоя на скалярное значение.
            eta_derivative   : function 
                Производная функции eta
            iterations       : int
                Количество итераций, выполняемых во время транса
        """
        if filename != '':
            self.load_model(filename)
            np.random.RandomState(rstate)
        else:
            np.random.RandomState(rstate)

            self.U = Param(K,d,I)
            self.num_layers = K
            self.dimension = d
            self.num_images = I
            self.gradient_descent = gradient_descent_method(self, *gradient_descent_input)
            self.one = np.ones((I,1))
            self.Y = np.zeros((K+1,d,I))
            self.Z = np.zeros((d,1))
            
            self.get_dataset(data[0],data[1], data[2],  data[3])
            
            self.Y[0,:,:] = self.training_data
            self.c = self.training_labels
            self.sigma, self.sigma_derivative = activation_functions_list[0], activation_functions_list[1]
            self.eta, self.eta_derivative = activation_functions_list[2], activation_functions_list[3]
            self.iterations = num_iterations
    
    def get_dataset(self, X, y, X_train, y_train):
        """Функция загрузки набора данных для обучения и тестирования
        
        Parameters
        ----------
        
        X       : np.array
            test 'images'. shape : dimension x num_images
        y       : np.array
            test labels. shape : num_images x 1
        X_train : np.array
            train 'images'. shape : dimension x num_images
        y_train : np.array
            test labels. shape : num_images x 1
        """
        
        self.training_data = X_train
        self.training_labels = y_train
        self.validation_data = X
        self.validation_labels = y
        
    def cost_function(self):
        """Calculates the least square error of the current output form the network."""
        return 1/2*(np.linalg.norm(self.Z-self.c))**2
    
    def projection(self,Y):
        """Calculates output. Uses the last values of Y and send them through the outputlayer."""
        return self.eta(Y[-1].T@self.U.omega + self.U.mu)
    
    def forward_prop(self,Y):
        """Forward propagation. Calculates all values of Y based on weights and biases."""
        for i in range(self.num_layers):
            Y[i+1,:,:] = Y[i,:,:] +  self.steplength*self.sigma(self.U.weight[i,:,:]@Y[i,:,:] + self.U.bias_vec[i])  
        
    def train(self, h=0.1, tau=0.01):
        """Training of the network
        Parameters
        ----------
        h : float
            steplength
        """
        
        # REVIEWER'S NOTE, REMOVE BEFORE PUBLISH!
        # Empty parameter list in docstring.
        self.cost_per_iter = np.zeros(self.iterations-1)
        self.validation_cost_per_iter = np.zeros(self.iterations-1)
        self.steplength = h
        self.tau = tau

        for i in tqdm(range(self.iterations)):  # tqdm creates a progressbar
            
            self.forward_prop(self.Y)
            
            self.Z = self.projection(self.Y)
            self.U.calculate_P_K(self)
            
            self.gradient_descent.update_first(self)
            self.U.calculate_P(self)
            
            self.gradient_descent.update_second(self)
            
            self.Z = self.projection(self.Y)
            
            self.gradient_descent.update_params(self,i)
            
            self.Z = self.projection(self.Y)
            
            if i != 0:
                self.cost_per_iter[i - 1] = self.cost_function()
                pred = self.predict(self.validation_data, integers = False)
                self.validation_cost_per_iter[i-1] = 1/2*(np.linalg.norm(pred-self.validation_labels))**2

                
    def predict(self, X, integers=True):
        """Makes a prediction based on current weights and biases.
        
        Parameters
        ----------
        X : np.array
            Input to network
            
        Returns
        -------
        prediction : np.array
            Output of the network
        """
        Y_pred = np.zeros((self.num_layers+1,self.dimension,len(X[0])))
        Y_pred[0,:,:] = X
        self.forward_prop(Y_pred)
        
        prediction = self.projection(Y_pred)
        
        if integers == True:
            prediction[prediction>=0.5] = 1
            prediction[prediction<0.5] = 0
        return prediction

    
    def evolution(self, filename=""):
        """Function for plotting how the performance of the model evolves.
        
        Parameters 
        ----------
        
        filename : string
            default = "" : does not save figure, else filename
            specifies the name of the file to which the plot will be saved.
        """
        
        fig = plt.figure()
        plt.title(r"\textbf{Cost as a function of iteration}")
        
        plt.plot(
            np.arange(self.iterations-1),
            self.cost_per_iter,
            label = r"$\mathcal{J}(\mathbf{U}^{(j)})$"
        )
        
        plt.xlabel(r"$j$")
        plt.ylabel(r"$\mathcal{J}(\mathbf{U}^{(j)})$")
        plt.grid(ls ="--")
        
        plt.yscale("log")  # Logarithmic scale of y-axis to better see how it evolves with j.
        
        plt.tight_layout()
        plt.legend()
        
        if filename != "":
            fig.savefig(filename)
        
    def compare_evolution(self, other, label1, label2, filename=""):
        """Function for plotting the performance of the model compared to another
        model.
        
        Parameters
        ----------
        other : Network
            trained network of the same type. Most meaningful to compare if it is 
            trained with the same amount of iterations.
            
        label1 : string
            label of self
        
        label2 : string
            label of other
        
        filename : string
            default = "" : does not save figure, else filename
            specifies the name of the file to which the plot will be saved.
        
        """
        fig = plt.figure()
        
        plt.title(r"\textbf{Cost as a function of iteration}")
        
        plt.plot(
            np.arange(self.iterations-1),
            self.cost_per_iter,
            label=r"$\mathcal{J}(\mathbf{U}^{(j)})_{\textup{%s}}$" %label1
        )
        plt.plot(
            np.arange(other.iterations-1),
            other.cost_per_iter,
            label=r"$\mathcal{J}(\mathbf{U}^{(j)})_{\textup{%s}}$" %label2
        )
        
        plt.legend()
        
        plt.xlabel(r"$j$")
        plt.ylabel(r"$\mathcal{J}(\mathbf{U}^{(j)})$")
        
        plt.grid(ls ="--")
        plt.yscale("log")  # Logarithmic scale of y-axis to see how it evolves with j better.
        
        fig.tight_layout()
        
        if filename != "":
            fig.savefig(filename)
    
    def accuracy(self, X, y):
        """Use a validation set {X,y} to check the accuracy of the network. 
        
        Parameters
        ----------
        X : np.array
            Validation set input
        y : np.array
            Validation set true value for output
        
        Returns
        -------
        accuracy : float
            The accuracy is the percentage of correct predictions
        """
        Y_test = self.predict(X)
        accuracy = np.sum(y == Y_test)/len(Y_test)
        return accuracy
    
    def variance(self, X, y):
        """Use a validation set {X,y} to check the variance of the network. 
        
        Parameters
        ----------
        X : np.array
            Validation set input
        y : np.array
            Validation set true value for output
        
        Returns
        -------
        variance : float
            The variance is 1/(n-1) sum((prediction - true value)^2)
        """
        Y_test = self.predict(X, integers=False)
        variance = 1/(len(y)-1)*np.sum(np.square(Y_test-y))
        return variance
    
    def confusion_matrix(self, X, y, label=""):
        """Use a validation set {X,y} to test if the model is biased. 

        Parameters
        ----------
        X : np.array
            Validation set input
        y : np.array
            Validation set true value for output
        label : string
            Label to append to figure title
        """
        Y_test = self.predict(X)
        
        Y_test = Y_test[:,0]
        y      = y[:,0]  
        Y_test = np.array([int(y_i) for y_i in Y_test])
        
        TP = np.sum(Y_test&y)         # True positives
        TN = np.sum((1-Y_test)&(1-y)) # True negatives
        FP = np.sum(Y_test&(1-y))     # False positives
        FN = np.sum((1-Y_test)&y)     # False negatives
        
        # Normalising data
        
        tn = TN/(TN + FN)
        fn = FN/(TN + FN)
        fp = FP/(FP + TP)
        tp = TP/(FP + TP)
        
        M = np.array([[tn,fn],
                      [fp,tp]])
        
        fig, ax = plt.subplots()
        
        plt.title(f"Confusion matrix{' - ' + label if label else ''}")
        
        # Using the seaborn heatmap-function
        sns.heatmap(M, annot=True, 
                    square = True, 
                    xticklabels=[0,1], 
                    yticklabels=[0,1],
                    vmax = 1,
                    vmin = 0
                   )
        
        ax.set_xlabel("Predicted value")
        ax.set_ylabel("Actual value")
        
        plt.tight_layout()
        
    
    def visualize_layers(self):
        """Visualizes how the input data is transformed through the layers of 
        the network. This function is only sensible to use when the data
        is two-dimensional, and the number of nodes in each layer is the same
        as the input dimension.
        """
        height = int(np.ceil((self.num_layers + 1)/4))  # Number of columns of plot
        
        fig, ax = plt.subplots(ncols=4, nrows=height, figsize=(14, 3 * height))
        fig.suptitle(r"\textbf{Grid transformations progression}", fontsize=26)
        
        for i in range(self.num_layers + 1):
            k = i // 4  # Row-index
            j = i - k * 4   # Column-index
            
            ax[k,j].scatter(x=(self.Y[i,:,:])[0,:], 
                            y=(self.Y[i,:,:])[1,:], 
                            s=1, 
                            c=self.c.flatten(), 
                            cmap='bwr')
            ax[k,j].axis([-1.2, 1.2, -1.2, 1.2])
            ax[k,j].axis('square')
            ax[k,j].axis("off") #  Removing frame of axis
        
        for i in range(height * 4 - self.num_layers + 1):
            # Deleting unused subplots    
            ax[height-1,i].axis("off")

        plt.tight_layout()
        plt.show()
        
    def training_vs_validation_error(self, filename=""):
        """Compares training error with validation error, using a validation set {X, y}
        
        Parameters
        ----------
        X : np.array
            Validation set input
        y : np.array
            Validation set true value for output
        filename : string
            default = "" : does not save figure, else filename
            specifies the name of the file to which the plot will be saved.
        """
        
        fig = plt.figure()
        plt.title(r"\textbf{Cost as a function of iteration. Validation vs training}")
        
        plt.plot(
            np.arange(self.iterations-1),self.cost_per_iter,
            label="Training cost"
        )
        plt.plot(
            np.arange(self.iterations-1),self.validation_cost_per_iter,
            label="Validation cost"
        )
        
        plt.xlabel(r"$j$")
        plt.ylabel(r"$\mathcal{J}(\mathbf{U}^{(j)})$")
        plt.grid(ls ="--")
        
        plt.yscale("log")  # Logarithmic scale of y-axis to see how it evolves with j better.
        
        plt.tight_layout()
        plt.legend()
        
        if filename != "":
            fig.savefig(filename)
        
        
    
    def save_model(self, filename):
        """Saves a trained model. Saves all values neccesary to make predictions with the model. 

        Parameters
        ----------
        filename : string
            Name of binary-file to ave model to.
        """
        # Create sub dictionaries to structre the data.
        parameters = {}
        parameters['weight'] = self.U.weight
        parameters['bias_vec'] = self.U.bias_vec
        parameters['mu'] = self.U.mu
        parameters['omega'] = self.U.omega

        dimensions = {}
        dimensions['K'] = self.num_layers
        dimensions['d'] = self.dimension
        dimensions['I'] = self.num_images
        dimensions['iterations'] = self.iterations 

        functions = {}
        functions['sigma'] = self.sigma
        functions['sigma_derivative'] = self.sigma_derivative
        functions['eta'] = self.eta
        functions['eta_derivative'] = self.eta_derivative

        # Collect all dictionaries in one.
        network_dict = {}
        network_dict['parameters'] = parameters
        network_dict['dimensions'] = dimensions
        network_dict['functions'] = functions

        # Save dictionary as binay file.
        # 'w' for write, 'b' to open as binary file (text file is default).
        with open(filename, 'wb') as outfile: 
            pickle.dump(network_dict, outfile, pickle.HIGHEST_PROTOCOL)

    def load_model(self, filename):
        """Loads up a trained model. Sets all valules neccesary to make predictions with the model. 

        Parameters
        ----------

        filename : string
            Name of binary-file to load model form. The file must contain a dictionary.
        """
        # Open file as binary file.
        # 'r' for read, add 'b' to open the file as a binary file (default is text file)
        file_to_read = open(filename, "rb")
        print('file', file_to_read)

        # Converts binary file to.
        network_dict = pickle.load(file_to_read)
        print('dict', network_dict)

        self.num_layers = network_dict['dimensions']['K']
        self.dimension = network_dict['dimensions']['d']
        self.num_images = network_dict['dimensions']['I']
        self.iterations = network_dict['dimensions']['iterations']

        # Initalize U with random values.
        self.U = Param(self.num_layers,self.dimension,self.num_images)

        # Set the right values for U.
        self.U.weight = network_dict['parameters']['weight']
        self.U.bias_vec = network_dict['parameters']['bias_vec']
        self.U.mu = network_dict['parameters']['mu']
        self.U.omega = network_dict['parameters']['omega']

        self.sigma = network_dict['functions']['sigma']
        self.sigma_derivative = network_dict['functions']['sigma_derivative']
        self.eta = network_dict['functions']['eta']
        self.eta_derivative = network_dict['functions']['eta_derivative']


### Набор данных

В этом блокноте мы протестировали сеть на простой и легкодоступной форме набора данных `sklearn`. Набор данных загружается с помощью вызова `sklearn.datsets.make_moons(n)`, где $n$ - количество точек данных. Набор данных состоит из 2-мерных входных данных, которые представляют точки в сетке с соответствующими метками. Вызов функции возвращает две переменные, $X$ и $y$, где $X$ содержит точки данных, а $y$ содержит связанные метки. Ниже мы построили набор данных для $n=400$ и раскрасили точки в соответствии с метками.

Функция вернет точки в двух полукругах в $\mathbb{R}^2$, помеченные $1$ или $0$ в зависимости от их координат. Мы решили присвоить цвет _blue_ экземплярам с меткой $0$ и _red_ экземплярам с меткой $1$. Чтобы сделать проблему менее тривиальной, мы добавляем к данным шум, из-за которого точки лежат немного в стороне от полукруга, к которому они принадлежат. Данные с шумом и без шума показаны ниже. Функция по умолчанию возвращает точки в случайном порядке. Значение `random_state` устанавливается для того, чтобы функция каждый раз возвращала точки в одном и том же порядке.

In [ ]:
from sklearn import datasets

#Vizualization of dataset
X, y = datasets.make_moons(400,random_state = 42, noise = 0)

blue = X[y==0]
red  = X[y==1]

plt.title(r"\textbf{Data without noise}")
plt.plot(blue[:,0],blue[:,1],"bo")
plt.plot(red[:,0],red[:,1],"ro")
plt.show()

In [ ]:
#Vizualization of dataset
X, y = datasets.make_moons(400,random_state = 42, noise = 0.15)

blue = X[y==0]
red  = X[y==1]

plt.title(r"\textbf{Data with noise}")
plt.plot(blue[:,0],blue[:,1],"bo")
plt.plot(red[:,0],red[:,1],"ro")
plt.show()

## Протестируем сеть

Пришло время протестировать сеть. Цель будет состоять в том, чтобы классифицировать, принадлежит ли точка синему или красному полукругу в наборе данных. Мы протестируем как градиентный спуск Адама, так и ванильный градиентный спуск, и поэтому мы создадим две отдельные сети.

Во время обучения сети мы также проверяем, как она работает с другим набором данных. Поэтому мы делаем два набора данных _separate_.

In [ ]:
I = 1000  # Количество точек данных

X_train, y_train = datasets.make_moons(I, random_state=100, noise=0.1)      
X_validate, y_validate = datasets.make_moons(I, random_state=30, noise=0.1)  

y_train = np.reshape(y_train, (I,1))  # Change the shape to adapt it to the network.
y_validate = np.reshape(y_validate, (I,1))

netAdam = Network(
    K=15,        # number of layers
    d=2,         # dimension of input
    I=I,         # number of datapoints
    num_iterations=5000,
    activation_functions_list=[tanh, tanh_derivative, sigmoid, sigmoid_derivative],
    gradient_descent_method=Adam,
    gradient_descent_input=[], # using default values of the parameters
    data=[X_validate.T, y_validate, X_train.T, y_train],
)


netVanilla = Network(
    K=15,        # number of layers
    d=2,         # dimension of input
    I=I,         # number of datapoints
    num_iterations=5000,
    activation_functions_list=[tanh, tanh_derivative, sigmoid, sigmoid_derivative],
    gradient_descent_method=Plain_vanilla,
    gradient_descent_input=[], # using default values of the parameters
    data=[X_validate.T, y_validate, X_train.T, y_train],
)

# If you wish to save a model, or load a model, this is how its done:
# netAdam.save_model('test')
# AdamCopy = Network(filename = 'test')

In [ ]:
netVanilla.train()
netAdam.train()

Чтобы визуализировать, как стоимость развивается в зависимости от индекса итерации, мы используем функцию `Network.training_vs_validation_error()`.

In [ ]:
netVanilla.training_vs_validation_error()

In [ ]:
netAdam.training_vs_validation_error()

Как показано выше, стоимость валидации в большинстве случаев достигнет точки, за которой она существенно не улучшит классификацию. Стоимость валидации может даже увеличиться, если мы пойдем дальше, хотя стоимость обучения продолжает снижаться. В идеале следует прекратить обучение с минимальными затратами на проверку, так как дальнейшее обучение сети приведет к ее переобучению(_overfitting_). То есть вместо того, чтобы улавливать суть структуры данных, она специализируется на конкретных данных, которые ей были даны. Эвристически сеть обманывает себя, рассматривая случайный шум в данных как часть структуры, которую она должна предсказать. Это приводит к тому, что сеть работает очень хорошо с обучающими данными, но хуже с тестирующими данными. 

В качестве иллюстрации этого рассмотрим задачу аппроксимации многочлена множеством точек. Если у нас есть N точек, которые мы хотим аппроксимировать нашим полиномом, мы знаем, что нам гарантировано, что существует полином степени не более N−1, который проходит через все точки. Однако это не обязательно тот полином, который наиболее близко напоминает истинную модель. Другой крайний случай-когда модель слишком проста, и мы называем ее недоученной (_underfitting_).

![overfitting](images/overfitting_fig.png "overfitting")

### Визуализация обучения

Основная цель обучения - найти значения параметров $\mathbf{U}$, которые минимизируют $\mathcal{J}$. Чтобы увидеть, работает ли обучение так, как ожидалось, мы можем построить график $\mathcal{J}(\mathbf{U}^{(j)})$, чтобы увидеть, как изменяется стоимость в зависимости от итераций $j$. Мы используем функцию `Network.compare_evolution`, реализованную в классе `Network`, для визуализации обучения для обеих сетей.

In [ ]:
netVanilla.compare_evolution(netAdam, "Vanilla", "Adam")

Как видно из приведенного выше графика, сеть, обученная с помощью алгоритма Adam, обучается намного быстрее, чем сеть, обученная с помощью алгоритма Plain-Vanilla. Тем не менее, поскольку задача настолько проста, погрешность обеих сетей через некоторое время стагнирует. Поэтому обучение Adam-сети за $5000$ итераций в данном случае является довольно чрезмерным, так как это не повышает производительность. Это также видно ниже, где мы сравниваем, как работают две сети в *матрице ошибок*.

На графике ниже показано положение красных и синих точек данных после начала преобразования через каждый из слоев сети. Из приведенных ниже визуализаций мы видим, что сеть учится скручивать и распутывать две спирали, чтобы было легко сделать классификацию, что она просто делает, рисуя прямую линию в $\mathbb{R}^2$, которая отделяет синий кусок от красного. Понимание, полученное в результате этой искусственно простой проблемы, действительно может быть перенесено на более сложные данные более высокого измерения. Проблема в том, что в задачах с более высокими измерениями невозможно визуализировать этот процесс с такой же легкостью. Если бы, например, мы должны были двоично классифицировать точки в $N$-мерном пространстве, обученная сеть отображала бы точки в $N$-мерном пространстве через слои, и в конечном итоге она нарисовала бы гиперплоскость $N-1$, разделяющую два класса на две области пространства. 

In [ ]:
netAdam.visualize_layers()

С точки зрения сети, нет ничего особенного в классификации точек в спиралях по сравнению с любой другой точкой на плоскости. Если бы мы классифицировали область вокруг спиралей, мы ожидаем, что классификация будет хорошей вблизи каждой спирали и в меньшей степени между ними. Это именно то, что мы наблюдаем ниже. 

In [ ]:
N = 1000

y = np.linspace(-1,1,N)
x = np.linspace(-1,2,N)

xx, yy = np.meshgrid(x,y)

plane = np.reshape((xx,yy), (2, N**2))

Z = netAdam.predict(plane, integers = False)

plt.contourf(x, y, Z.reshape((N,N)), cmap='seismic', levels = 100)
plt.contour(x, y, Z.reshape((N,N)), levels=1, colors='k')
plt.xlim([-1,2])
plt.ylim([-1,1])
plt.show()

### Точность сети

Несмотря на то, что важно, чтобы сеть хорошо работала во время обучения, именно тесты, выполненные с проверочным набором, действительно говорят вам, насколько хороша сеть. Здесь вы тестируете сеть на новых данных и видите, насколько хорошо она преформируется в целом. Существует множество способов тестирования сети, но мы выбрали три простых метода, которые могут быть реализованы для большинства проблем. Сначала мы проверяем точность сети, т.е. Какой процент догадок верен. Если точность близка к $50 \%$ для двоичной классификации, это означает, что сеть не смогла узнать связь между входными и выходными данными, и с тем же успехом можно использовать случайные предположения. 

In [ ]:
#Method beloning to network
#self = Network

def accuracy(self, X, y):
    """Use a validation set {X,y} to check the accuracy of the network. 

    Parameters
    ----------
    X : np.array
        Validation set input
    y : np.array
        Validation set true value for output

    Returns
    -------
    accuracy : float
        The accuracy is the percentage of correct predictions
    """
    Y_test = self.predict(X)
    accuracy = np.sum(y == Y_test)/len(Y_test)
    return accuracy

In [ ]:
n = 10000 
X_validate, c_validate = datasets.make_moons(n,noise = 0.15)

print("Accuracy Adam:    %.3f" %netAdam.accuracy(X_validate.T, np.array([c_validate]).T))
print("Accuracy Vanilla: %.3f" %netVanilla.accuracy(X_validate.T, np.array([c_validate]).T))

Во-вторых, мы проверяем дисперсию сети. Здесь мы видим, насколько сеть уверена в своих догадках. В идеале сеть будет выводить либо $1$, либо $0$ для двух категорий, но на самом деле она возвращает десятичное число между $1$ и $0$. Дисперсия - это мера того, насколько близки или далеки от истинного значения предположения. Чем меньше дисперсия, тем лучше.

In [ ]:
#Function beloning to network
#self = Network

def variance(self, X, y):
    """
    Use a validation set {X,y} to check the variance of the network. 

    Parameters
    ----------
    X : np.array
        Validation set input
    y : np.array
        Validation set true value for output

    Returns
    -------
    variance : float
        The variance is 1/(n-1) sum((prediction - true value)^2)
    """
    Y_test = self.predict(X, integers=False)
    variance = 1/(len(y)-1)*np.sum(np.square(Y_test-y))
    return variance

In [ ]:
print("Variance Adam:    %.3e" %netAdam.variance(X_validate.T, np.array([c_validate]).T))
print("Variance Vanilla: %.3e " %netVanilla.variance(X_validate.T, np.array([c_validate]).T))

Третий тест - это матрица ошибок. Это квадратная матрица $n\times n$ , где $n$ - количество классов, которые она предсказывает. Строки представляют истинное значение класса, а столбцы-предполагаемое значение. Элемент $(i,j)$ матрицы - (нормализованное) число предполагаемых экземпляров категории $j$, которые имеют истинное значение $i$. Идеальной классификацией тогда была бы единичная матрица. Ниже мы построим матрицы ошибок  для проверки в двух сетях. В случае двоичной классификации мы можем записать элементы как истинные отрицательные (TN), истинные положительные (TP), ложные отрицательные (FN) и ложные положительные (FP) следующим образом

$$
    \begin{bmatrix}
        \frac{\text{TN}}{\text{N}} & \frac{\text{FN}}{N} \\
        \frac{\text{FP}}{\text{P}} & \frac{\text{TP}}{P}
    \end{bmatrix},
$$

где P-количество положительных экземпляров (категория 1), а N-количество отрицательных экземпляров (категория 0). 

In [ ]:
bias_test_vanilla = netVanilla.confusion_matrix(X_validate.T, np.array([c_validate]).T)
bias_test_adam = netAdam.confusion_matrix(X_validate.T, np.array([c_validate]).T)

## Комментарии

### Несбалансированный набор данных

В этом блокноте мы использовали очень идеализированный набор данных, который всегда имеет равное количество точек данных для каждого класса. При использовании реальных данных это происходит редко. Если набор данных имеет большой избыточный вес одного из классов, есть большая вероятность, что сеть будет предвзято относить большинство объектов к этому классу, поскольку это, как правило, является хорошим решением во время обучения. Это одна из многих проблем, с которыми вы можете столкнуться при работе с реальными наборами данных. Ниже приводится иллюстрация того, что может произойти в случае несбалансированного набора данных. В этом примере мы включаем только $10$ синих точек данных и $250$ красных, чтобы создать искусственно несбалансированный набор данных.

In [ ]:
I = 500  # Number of datapoints

X_train, y_train = datasets.make_moons(I, random_state=100, noise=0.1)      
X_validate, y_validate = datasets.make_moons(I, random_state=30, noise=0.1)  

# Removing some of the blue datapoints 

blue = y_train == 0
red  = y_train == 1
y_blue = y_train[blue]
X_blue = X_train[blue]
y_red  = y_train[red]
X_red  = X_train[red]

y = y_blue[:10]
X = X_blue[:10]

X_train_new = np.concatenate((X,X_red), axis = 0 )
y_train_new = np.concatenate((y,y_red), axis = 0 )

y_train_new = np.reshape(y_train_new, (260,1))  # Change the shape to adapt it to the network.
y_validate = np.reshape(y_validate, (I,1))

imbalancedNet = Network(
    K=15,          # number of layers
    d=2,           # dimension of input
    I=260,         # number of datapoints
    num_iterations=1000,
    activation_functions_list=[tanh, tanh_derivative, sigmoid, sigmoid_derivative],
    gradient_descent_method=Plain_vanilla,
    gradient_descent_input=[], # using default values of the parameters
    data=[X_validate.T, y_validate, X_train_new.T, y_train_new],
)

In [ ]:
imbalancedNet.train()

In [ ]:
n = 10000 
X_validate, c_validate = datasets.make_moons(n,noise = 0.15)

bias_test_imbalanced = imbalancedNet.confusion_matrix(X_validate.T, np.array([c_validate]).T)

### Обучение, валидация и набор тестов

В этом блокноте мы использовали только обучающие и проверочные наборы. Обычно также используется тестовый набор. Очевидно, что обучающий набор используется во время обучения сети. Набор проверки используется для проверки производительности модели *при разработке* сети. Это может быть, например, для поиска наилучших значений количества слоев, итераций и нейронов в сети. Когда вы довольны всей сетью и всеми ее параметрами, вы проводите заключительный тест с набором тестов. Это делается для того, чтобы изменения, внесенные в ходе разработки, также подходили для тестовых данных. 

# Следующий шаг 

Несмотря на то, что набор данных в этом блокноте является идеализированным, теория может быть применена к широкому кругу проблем. Создание модели машинного обучения с нуля может быть интересным и увлекательным, но для более эффективных и сложных моделей мы рекомендуем попробовать некоторые пакеты, доступные для Python:


- <a href="https://www.tensorflow.org/overview">Getting started with Tensorflow</a>.
- <a href="https://pytorch.org/tutorials/">Getting started with PyTorch</a>.
- <a href="https://scikit-learn.org/stable/getting_started.html">Getting started with Scikit-learn</a>.

<a href = "https://pandas.pydata.org/docs/getting_started/index.html"> Пакет Pandas </a> также очень полезен при работе с большими объемами данных, и большинство пакетов машинного обучения в Python совместимы с фреймами данных Pandas.


Полносвязные нейронные сети - очень популярный тип машинного обучения, но существует множество различных моделей, как в категории нейронных сетей, так и моделей, использующих совершенно разные структуры.

- <a href = "https://machinelearningmastery.com/gentle-introduction-long-short-term-memory-networks-experts/"> Long short term memory (LSTM) networks </a> может использоваться, например, для распознавания речи и специализируется на поиске паттернов во времени.
- <a href =  "https://www.geeksforgeeks.org/what-is-reinforcement-learning/"> Reinforcement learning </a> это тип неконтролируемого метода машинного обучения, который использует вознаграждения во время обучения вместо истинных значений для каждой точки данных.
- <a href =  "https://towardsdatascience.com/understanding-random-forest-58381e0602d2"> Random forest </a> это метод машинного обучения, который облегчает определение того, какие переменные являются наиболее важными для классификации или регрессии.
- <a href =  "https://towardsdatascience.com/introduction-to-logistic-regression-66248243c148"> Logistic regression </a> это двоичный классификатор, который, как известно, прост в реализации и взаимопроникновении, и может быть хорошим для начала, если вы заинтересованы в статистических данных, лежащих в основе машинного обучения.
- <a href =  "https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47"> Support vector machine </a> может обеспечить хорошую точность как для задач классификации, так и для задач регрессии с меньшей вычислительной мощностью, чем, например, нейронные сети.
- <a href =  "https://towardsdatascience.com/understanding-k-means-clustering-in-machine-learning-6a6e67336aa1"> K-means </a> это неконтролируемый метод поиска шаблонов в (немаркированном) наборе данных.

Сноски <br>
$^1$ ANN также может иметь менее трех слоев; Однослойный прецептрон состоит только из выходного слоя. <br>
$^2$ Не во всех сетях есть нейроны, которые выводят значения от 0 до 1. Именно диапазон функции активации определяет масштабирование выходных данных нейронов. <br>
$^3$ Или любая другая функция активации. <br>

# References 
 <a name="biological_net">[1]</a> _Neural network_ https://en.wikipedia.org/wiki/Neural_network <br>
 <a name="activation_functions">[2]</a> _Understanding Activation Functions in Neural Networks_ https://medium.com/the-theory-of-everything/understanding-activation-functions-in-neural-networks-9491262884e0 <br>
 <a name="ann_training">[3]</a> _Activation functions in neural networks_ https://www.geeksforgeeks.org/activation-functions-neural-networks/ <br>
 <a name="ReLU">[4]</a> _A Practical Guide to ReLU_ https://medium.com/@danqing/a-practical-guide-to-relu-b83ca804f1f7 <br>
 <a name="initialization_of_parameters">[5]</a> _Weight Initialization Techniques in Neural Networks_ https://towardsdatascience.com/weight-initialization-techniques-in-neural-networks-26c649eb3b78 
 
 The original report about the ResNet: <br>
 _Deep Residual Learning for Image Recognition_ https://arxiv.org/abs/1512.03385
 
 https://habr.com/ru/post/474084/ 
 https://habr.com/ru/post/478790/

# Приложение

## Вывод градиентных формул

В тексте мы вывели градиенты $\mathcal{J}$ по отношению к $\mu$ и $\omega$. Здесь мы выведем два других градиента, и с этой целью мы представим немного новой нотации, чтобы более элегантно обрабатывать выражения.

Пусть $H : \mathbb{R}^m \to \mathbb{R}^n$ - функция переменных $m$, а $\mathbf{x},\boldsymbol{\delta}$ - векторы в $\mathbb{R}^m$. Направленная производная $H$ в $\mathbf{x}$ в направлении $\boldsymbol{\delta}$ как 

$$
\frac{\text{d}}{\text{d} \epsilon} \biggr\lvert_{\epsilon = 0} H(\mathbf{x} + \epsilon \boldsymbol{\delta}) = \nabla H (\mathbf{x}) \cdot \boldsymbol{\delta} = \left\langle \frac{\partial H}{\partial \mathbf{x}} (\mathbf{x}), \boldsymbol{\delta} \right\rangle.
$$

Рассмотрим изменение $\mathcal{J}$ в направлении $\delta W_k$. Поскольку $\mathcal{J}$ зависит только косвенно от $W_k$, через $\mathbf{Y}_K$, мы сначала рассмотрим изменение вдоль направления $\delta \mathbf{Y}_K$ 

\begin{equation}\label{eq:del_J}
    \delta \mathcal{J} = \frac{\text{d}}{\text{d} \epsilon} \biggr\lvert_{\epsilon = 0} \mathcal{J}(\mathbf{Y}_K + \epsilon \delta \mathbf{Y}_K) = \left\langle \frac{\partial \mathcal{J}}{\partial \mathbf{Y}_K}, \delta \mathbf{Y}_K \right\rangle. \quad (1)
\end{equation}

Теперь введем $\mathbf{P}_K := \frac{\partial \mathcal{J}}{\partial \mathbf{Y}_K}$. Так как у нас есть 

\begin{equation}\label{eq:transf}
    \mathbf{Y}_K = \mathbf{Y}_{K-1} + h \sigma{\left( W_{K-1} \mathbf{Y}_{K-1} + b_{K-1} \right)},\quad (2)
\end{equation}

изменение $\delta \mathbf{Y}_K$ равно 

$$
    \delta \mathbf{Y}_K = \left( I +  h \frac{\partial \sigma}{\partial \mathbf{Y}} (W_{K-1} \mathbf{Y}_{K-1} + b_{K-1}) \right) \delta \mathbf{Y}_{K-1}.
$$

Мы можем вставить это в уравнение (1) и использовать тот факт, что $\langle v, Aw \rangle = \langle A^T v, w \rangle$ для произвольных векторов $v\in \mathbb{R}^n, w \in \mathbb{R}^m$ и матрицы $A \in \mathbb{R}^{n\times m}$. Следовательно, 

\begin{align*}
    \delta \mathcal{J} &= \left\langle \mathbf{P}_K, \left( I +  h\frac{\partial \sigma}{\partial \mathbf{Y}} (W_{K-1} \mathbf{Y}_{K-1} + b_{K-1}) \right) \delta \mathbf{Y}_{K-1} \right\rangle \\
    &= \left\langle \mathbf{P}_K +  h\left[\frac{\partial \sigma}{\partial \mathbf{Y}} (W_{K-1} \mathbf{Y}_{K-1} + b_{K-1})\right]^T \mathbf{P}_K, \delta \mathbf{Y}_{K-1} \right\rangle.\quad (3)
\end{align*}

Поэтому мы определяем в целом 

\begin{equation}\label{eq:P_k-1_simple}
    \mathbf{P}_{k-1} = \mathbf{P}_k + h \left[ \frac{\partial \sigma}{\partial \mathbf{Y}} (W_{k-1} \mathbf{Y}_{k-1} + b_{k-1})\right]^T \mathbf{P}_k.\quad (4)
\end{equation}

 то есть 

$$
\mathbf{P}_{k-1} = \frac{\partial \mathcal{J}}{\partial \mathbf{Y}_{k-1}} = \mathbf{P}_k +  h W_{k-1}^{T} \cdot \left[ \sigma' \left(W_{k-1} \mathbf{Y}_{k-1} + b_{k-1} \right) \odot \mathbf{P}_{k}\right].
$$

Таким образом, мы пришли к $\delta \mathcal{J} = \langle \mathbf{P}_{K-1},\delta \mathbf{Y}_{K-1} \rangle$. Продолжая аргументацию индуктивно, мы в конечном итоге найдем 

\begin{equation}\label{eq:del_J_compact}
    \delta \mathcal{J} = \langle \mathbf{P}_{k+1},\delta \mathbf{Y}_{k+1} \rangle.\quad (5)
\end{equation}

Поскольку $\mathbf{Y}_{k+1} = \mathbf{Y}_{k} + h \sigma{\left( W_{k} \mathbf{Y}_{k} + b_{k} \right)},$ мы можем написать 

\begin{equation}
    \delta \mathbf{Y}_{k+1} = h \frac{\partial \sigma}{\partial W} (W_{k} \mathbf{Y}_{k} + b_{k}) \delta W_k,
\end{equation}

что можем подставить в уравнение (5), чтобы получить 

$$
    \delta \mathcal{J} = \left\langle h \left[ \frac{\partial \sigma}{\partial W} (W_{k} \mathbf{Y}_{k} + b_{k})\right]^T \mathbf{P}_{k+1}, \delta W_k \right\rangle.
$$

Теперь мы можем заключить, что градиент $\mathcal{J}$ относительно $W_k$ задается как 

\begin{equation}\label{eq:delW}
    \frac{\partial \mathcal{J}}{\partial W_k} = h \left[ \frac{\partial \sigma}{\partial W} (W_{k} \mathbf{Y}_{k} + b_{k})\right]^T \mathbf{P}_{k+1}. 
\end{equation}

Заметим, что $\mathbf{P}_k$ необходим для вычисления $\mathbf{P}_{k-1}$, поэтому мы сначала вычисляем $\mathbf{P}$ для $k = K$, а затем переходим назад для всех $k < K$. Совершенно аналогичным образом мы получаем 

\begin{equation}\label{eq:delb}
     \frac{\partial \mathcal{J}}{\partial b_k} = h \left[ \frac{\partial \sigma}{\partial b} (W_{k} \mathbf{Y}_{k} + b_{k})\right]^T \mathbf{P}_{k+1}.
\end{equation}

## Явная формула для $\mathbf{P}_K$

Чтобы получить $\frac{\partial \mathcal{J}}{\partial \mathbf{Y}_K}$, мы рассмотрим следующее (и заметим, что мы подавляем индекс $K$)

$$
    \frac{\text{d}}{\text{d} \epsilon} \biggr\lvert_{\epsilon = 0} \mathcal{J}(\mathbf{Y} + \epsilon \delta \mathbf{Y}) = \frac{1}{2} \frac{\text{d}}{\text{d} \epsilon} \biggr\lvert_{\epsilon = 0} \left\langle
    \mathbf{Z}(\epsilon) - \mathbf{c}, \mathbf{Z}(\epsilon) - \mathbf{c}
    \right\rangle
    = \left\langle \mathbf{Z}(0) - \mathbf{c}, \mathbf{Z}^\prime(0) \right\rangle,
$$

где 

$$
\mathbf{Z}(\epsilon) = \eta\left( \left[\mathbf{Y} + \epsilon \delta \mathbf{Y})\right]^T \omega + \mu \mathbf{1}\right).
$$

По правилу цепочки мы находим 

$$
\mathbf{Z}^\prime(0) = \eta^\prime \left( \mathbf{Y}^T \omega + \mu \mathbf{1}\right) \delta \mathbf{Y}^T \omega.
$$

Из этого следует, что 

$$
    \left\langle \mathbf{Z}(0) - \mathbf{c}, \mathbf{Z}^\prime(0) \right\rangle = \left\langle \omega \left[ (\mathbf{Z} - \mathbf{c}) \odot \eta^\prime \left( \mathbf{Y}^T \omega + \mu \mathbf{1}\right) \right]^T, \delta \mathbf{Y}  \right\rangle,
$$

и выражение для $\frac{\partial \mathcal{J}}{\partial \mathbf{Y}_K}$ следует. 

## Вывод $\mathbf{P}_{k-1}$

Чтобы обосновать утверждение, что \eqref{eq:P_k-1} следует из \eqref{eq:P_k-1_simple}, достаточно показать, что 

$$
    \left[ \frac{\partial \sigma}{\partial \mathbf{Y}} (W_{k-1} \mathbf{Y}_{k-1} + b_{k-1})\right]^T \mathbf{P}_k = W_{k-1}^{T} \cdot \left[ \sigma' \left(W_{k-1} \mathbf{Y}_{k-1} + b_{k-1} \right) \odot \mathbf{P}_{k}\right].
$$

Далее мы будем подавлять индекс слоев и резервировать индекс для "компонентов". 

\begin{align*}
    \left\langle \left[ \frac{\partial \sigma}{\partial \mathbf{Y}} (\cdot) \right]^T \mathbf{P}, \delta \mathbf{Y} \right\rangle &= \left\langle \mathbf{P}, \frac{\partial \sigma}{\partial \mathbf{Y}} (\cdot) \delta \mathbf{Y} \right\rangle = \left\langle \mathbf{P}, \frac{\text{d}}{\text{d}\epsilon} \biggr\lvert_{\epsilon = 0} \sigma (W \left( \mathbf{Y} + \epsilon \delta\mathbf{Y} \right) + b ) \right\rangle \\
    &= \sum_{i,j,k} \mathbf{P}_{ij} \sigma^\prime \left( (W \mathbf{Y} + b)_{ij}\right) W_{ik} \delta \mathbf{Y}_{kj}  = \sum_{k,j} \delta \mathbf{Y}_{kj} \sum_{i} W_{ki}^{T} \left[ \mathbf{P} \odot \sigma^\prime(W\mathbf{Y} + b) \right]_{ij} \\
    &= \left\langle W^T \left[ \sigma^\prime (W\mathbf{Y} + b) \odot \mathbf{P} \right], \delta \mathbf{Y} \right\rangle.
\end{align*}

## Градиенты относительно $W_k$ и $b_k$

Чтобы получить окончательные уравнения для $\frac{\partial \mathcal{J}}{\partial W_k}$, мы начинаем с \eqref{eq:delW} и снова подавляем индекс слоя. Затем мы находим 

\begin{align*}
    \left\langle \frac{\partial \mathcal{J}}{\partial W}, \delta W \right\rangle &= \left\langle h \left[ \frac{\partial \sigma}{\partial W} (W \mathbf{Y} + b)\right]^T \mathbf{P}, \delta W \right\rangle = h \left\langle \mathbf{P}, \frac{\partial \sigma}{\partial W} (W \mathbf{Y} + b) \delta W \right\rangle\\
    &= h \left\langle \mathbf{P}, \frac{\text{d}}{\text{d}\epsilon} \biggr\lvert_{\epsilon = 0} \sigma ((W + \epsilon \delta W) \mathbf{Y} + b ) \right\rangle \\
    &= h \sum_{i,j,k} \mathbf{P}_{ij} \sigma^\prime ((W\mathbf{Y} + b)_{ij}) \delta W_{ik} \mathbf{Y}_{kj} = h \sum_{i,k} \delta W_{ik} \sum_{j} \left[ \mathbf{P} \odot \sigma^\prime (W\mathbf{Y} + b)\right]_{ij} \mathbf{Y}_{jk}^T \\
    &= h \left\langle \left[ \mathbf{P} \odot \sigma^\prime (W\mathbf{Y} + b) \right] \mathbf{Y}^T, \delta W \right\rangle.
\end{align*}

Это показывает окончательную формулу для $\frac{\partial \mathcal{J}}{\partial W_k}$. Мы опускаем доказательство формулы для $\frac{\partial \mathcal{J}}{\partial b_k}$, так как она получена очень похожим образом.